In [25]:
import pandas as pd
import sklearn
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from numpy import reshape
from sklearn import metrics
from collections import Counter 
from sklearn.impute import KNNImputer

In [26]:
data = pd.read_csv('train_jan06_task4bonus.csv')

data = data.dropna(axis=0, how='any')
data = pd.DataFrame(data)
data.isnull().values.any()

False

In [27]:
# split train and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=35)

In [28]:
# split features and labels
X_train = train_data.iloc[:, :14]
y_train = train_data.iloc[:, 14]

X_test = test_data.iloc[:, :14]
y_test = test_data.iloc[:, 14]

In [29]:
# tranform the labels to numbers to fit XGBoost
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [30]:
# check the imbalance of the data
type(y_train)

numpy.ndarray

In [31]:
# # use SMOTE to balance the data
# # deprecated: SMOTE + TomekLinks is not better than SMOTE
# from imblearn.over_sampling import SMOTE
# X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)

# # check the imbalance of the data again
# print(f"Training target statistics: {Counter(y_train)}")
# print(f"Testing target statistics: {Counter(y_test)}")
# type(y_train)

In [32]:
X_train

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13
5044,5.263536,2.888580,10.172511,5.964243,10.490974,11.204078,4.649965,4.530837,1.552903,4.464508,1.888097,4.333570,6.523952,5.604364
3524,3.847185,2.515811,8.486694,2.950003,9.772534,10.444022,10.186250,10.718132,9.171970,8.096505,1.314820,0.564652,6.305562,5.161377
2014,9.337414,3.632462,6.879610,7.067639,7.388772,9.269483,6.901187,9.384523,2.567512,2.395287,2.303567,5.345253,6.136224,6.561042
5364,10.610509,2.859282,8.300140,5.833840,9.401405,5.216764,17.606022,5.760159,7.173404,10.167979,0.234073,2.403171,3.770810,4.313245
5660,9.321674,5.444533,5.509881,5.923345,14.979760,10.022494,15.851459,2.436668,8.114382,11.528740,5.541106,5.905004,6.822657,6.469583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,8.544061,7.077767,4.207810,10.181200,12.435207,12.912851,7.333140,7.878006,5.052528,11.548862,5.862975,2.331614,5.529950,4.898901
4000,6.976476,8.400066,6.812281,6.053825,10.821075,7.187484,18.446845,7.878172,7.441846,0.626618,2.714128,0.599499,6.454127,4.026533
2380,12.074174,8.104266,4.460196,5.402023,10.020930,12.022437,14.326507,8.663517,10.162202,4.497671,4.377037,4.178784,6.084231,4.504453
3147,2.638430,7.150443,10.441031,5.389840,13.040506,10.942847,15.624016,4.966372,8.536939,1.195192,3.614993,5.357749,5.960302,5.592561


In [33]:
y_train

array([2, 2, 1, ..., 1, 2, 2])

In [34]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

# choose the best k features numbers
select_k = 5

selection = SelectKBest(mutual_info_classif, k=select_k).fit(X_train, y_train)

# show the selected features
''' Higher score means more important with the target 
    Lower score means less important with the target
'''

features = X_train.columns[selection.get_support()]
print(f"features: {features}")
print(f"scores: {selection.scores_}")
X_train = selection.fit_transform(X_train, y_train)
X_test = selection.transform(X_test)

features: Index(['feature2', 'feature3', 'feature4', 'feature5', 'feature6'], dtype='object')
scores: [0.0104691  0.         0.20580758 0.20021739 0.18678438 0.1673992
 0.11041241 0.         0.         0.         0.         0.01203053
 0.         0.        ]


In [35]:
# build XGBClassifier model
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.36)

# train data with train_data
# xgboostModel.fit(X_train, y_train)
xgboostModel.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.36, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [36]:
# print the accuracy
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  1.0
測試集:  0.7031539888682746
